In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import IPython.display as idis
import seaborn as sns
import copy
import scipy.stats as stat
import functools 
%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 300

In [2]:
#load data

Mimic_data = '/home/student/huy/mnt/mrAnKhuongData/storage/data/curated/all_hourly_data.h5'
vitals_data = pd.read_hdf(Mimic_data, 'vitals_labs')
interventions = pd.read_hdf(Mimic_data, 'interventions')
patients = pd.read_hdf(Mimic_data, 'patients')


In [3]:
abnormal_amittime_patients_index = functools.reduce(lambda x, y: x + [y] if patients.iloc[y,patients.columns.get_loc('admittime')] > patients.iloc[y,patients.columns.get_loc('intime')] else x +[], [i for i in range(0,patients.shape[0])] ,[])

abnormal_amittime_patients_index


for index in abnormal_amittime_patients_index:
    patients.iloc[index,patients.columns.get_loc('admittime')] = patients.iloc[index,patients.columns.get_loc('intime')]

patients.loc[patients['admittime'] > patients['intime']]


# Abnormal behavior: Some patients have admittime > dischtime

patients.loc[patients['admittime'] > patients['dischtime']]

#delete patients with abnormal behavior above

list_abnormal_admit_disch_gap = list(patients.loc[patients['admittime'] > patients['dischtime']].index.get_level_values('subject_id'))

patients = patients.drop(list_abnormal_admit_disch_gap)

#delete vitals lab of patients with abnormal behavior above
vitals_data = vitals_data.drop(list_abnormal_admit_disch_gap)

# check again in two table patients and vitals_data
print(patients.loc[patients['admittime'] > patients['dischtime']])
print(vitals_data.loc[list_abnormal_admit_disch_gap, :])


total_los_series = pd.Series(list(map(lambda x: round(x.total_seconds()/(24*60*60),4),(patients.iloc[:,patients.columns.get_loc('dischtime')] - patients.iloc[:,patients.columns.get_loc('admittime')]))))
patients['total_los'] = total_los_series.values


list_los_total_smaller_icu = patients.loc[(patients['total_los'] - patients['los_icu']) <= -0.5].index.get_level_values('subject_id')

patients = patients.drop(list_los_total_smaller_icu)
vitals_data = vitals_data.drop(list_los_total_smaller_icu)


# if los_icu > total_los => total_los := los icu

abnormal_icu_total_gap_index = functools.reduce(lambda x, y: x + [y] if patients.iloc[y,patients.columns.get_loc('total_los')] < patients.iloc[y,patients.columns.get_loc('los_icu')] else x +[], [i for i in range(0,patients.shape[0])] ,[])

abnormal_icu_total_gap_index


for index in abnormal_icu_total_gap_index:
    patients.iloc[index,patients.columns.get_loc('total_los')] = patients.iloc[index,patients.columns.get_loc('los_icu')]


# Filtering patients (subject_id) in vitals_data table based on patient in patients table 
# Get index in patients table
icustay_id_index = set(patients.index.get_level_values('icustay_id'))

#filtering vitals lab
vitals_data_filtered = vitals_data[vitals_data.index.get_level_values('icustay_id').isin(icustay_id_index)]


# Filtering vitals lab collected in the first 24h after first ICU admission and patients with icu_los at least window_time
# + gaptime (gap_time should be 1 to keep data)
# Gap time to avoid data leakage and make predicting model really useful in practice
# Also, Fil

WINDOW_SIZE = 24  
GAP_TIME = 1

Ys = patients[patients.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]

patients = patients[patients.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))]

vitals_data_filtered = vitals_data_filtered[(vitals_data_filtered.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) & (vitals_data_filtered.index.get_level_values('hours_in') < WINDOW_SIZE)]

Empty DataFrame
Columns: [gender, ethnicity, age, insurance, admittime, dischtime, deathtime, intime, outtime, los_icu, admission_type, first_careunit, mort_icu, mort_hosp, hospital_expire_flag, hospstay_seq, max_hours]
Index: []
Empty DataFrame
Columns: [(alanine aminotransferase, count), (alanine aminotransferase, mean), (alanine aminotransferase, std), (albumin, count), (albumin, mean), (albumin, std), (albumin ascites, count), (albumin ascites, mean), (albumin ascites, std), (albumin pleural, count), (albumin pleural, mean), (albumin pleural, std), (albumin urine, count), (albumin urine, mean), (albumin urine, std), (alkaline phosphate, count), (alkaline phosphate, mean), (alkaline phosphate, std), (anion gap, count), (anion gap, mean), (anion gap, std), (asparate aminotransferase, count), (asparate aminotransferase, mean), (asparate aminotransferase, std), (basophils, count), (basophils, mean), (basophils, std), (bicarbonate, count), (bicarbonate, mean), (bicarbonate, std), (bilir

In [4]:
patients[patients['total_los']>=60]

,,,gender,ethnicity,age,insurance,admittime,dischtime,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,max_hours,total_los
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,
145,198161,226841,F,WHITE,54.9939,Medicaid,2144-03-29 01:44:00,2144-07-14 16:15:00,NaT,2144-03-29 01:44:54,2144-04-03 15:51:53,5.5882,EMERGENCY,CCU,0,0,0,1,134,107.6049
223,105694,255617,M,WHITE,67.5093,Medicare,2157-05-01 05:19:00,2157-07-01 10:20:00,NaT,2157-05-01 05:20:41,2157-05-17 02:33:09,15.8837,EMERGENCY,MICU,0,0,0,1,381,61.2090
384,130196,291843,F,WHITE,68.2493,Medicare,2161-04-07 11:59:00,2161-06-06 14:21:00,NaT,2161-04-07 23:08:51,2161-04-10 18:58:22,2.8261,EMERGENCY,SICU,0,0,0,1,67,60.0986
408,173910,279910,M,WHITE,79.4813,Medicare,2188-10-27 21:25:00,2189-01-11 20:19:00,2189-01-11 20:19:00,2188-10-27 21:27:14,2188-10-30 19:54:32,2.9356,EMERGENCY,TSICU,0,1,1,1,70,75.9542
441,188256,290964,F,WHITE,41.2674,Private,2123-05-16 13:52:00,2123-08-05 14:00:00,NaT,2123-05-16 13:52:53,2123-08-05 17:52:26,81.1664,EMERGENCY,MICU,0,0,0,1,1947,81.1664
834,153730,246137,M,UNKNOWN/NOT SPECIFIED,76.0047,Medicare,2166-06-17 03:00:00,2166-09-12 11:30:00,2166-09-12 11:30:00,2166-06-20 11:14:00,2166-06-30 18:29:00,10.3021,EMERGENCY,CCU,0,1,1,1,247,87.3542
907,149649,223095,F,ASIAN,48.1233,Medicaid,2155-08-12 15:38:00,2155-10-18 12:14:00,NaT,2155-08-12 15:40:46,2155-09-05 18:22:12,24.1121,EMERGENCY,MICU,0,0,0,1,578,66.8583
1113,128609,257080,M,BLACK/AFRICAN AMERICAN,46.0454,Private,2106-12-07 17:19:00,2107-03-03 16:19:00,2107-03-03 16:19:00,2106-12-07 17:21:16,2106-12-15 16:30:18,7.9646,EMERGENCY,CCU,0,1,1,1,191,85.9583
1369,186547,208845,M,WHITE,70.8283,Medicare,2180-09-05 10:55:00,2180-11-15 15:10:00,NaT,2180-09-05 10:56:03,2180-09-09 16:44:14,4.2418,EMERGENCY,CCU,0,0,0,1,101,71.1771


In [5]:
vitals_data_filtered.iloc[vitals_data_filtered.index.get_level_values('subject_id') == 145]

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
145        198161  226841     0                             1.0  97.0  NaN   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   
                              5                             0.0   NaN  NaN   
                              6                             2.0  76.0  0.0   
                              7                             0.0   NaN  NaN   
                              8                             0.0   NaN  NaN   
                              9                             0.0   NaN  NaN   
                              10                            0.0   NaN  NaN   
                              11                            0.0   NaN  NaN   
                              12                            0.0   NaN  NaN   
                              13                            0.0   NaN  NaN   
                              14                            0.0   NaN  NaN   
                              15                            0.0   NaN  NaN   
                              16                            0.0   NaN  NaN   
                              17                            0.0   NaN  NaN   
                              18                            0.0   NaN  NaN   
                              19                            0.0   NaN  NaN   
                              20                            0.0   NaN  NaN   
                              21                            0.0   NaN  NaN   
                              22                            0.0   NaN  NaN   
                              23                            0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
145        198161  226841     0            0.0  NaN  NaN             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   
                              5            0.0  NaN  NaN             0.0  NaN   
                              6            3.0  3.5  0.0             0.0  NaN   
                              7            0.0  NaN  NaN             0.0  NaN   
                              8            0.0  NaN  NaN             0.0  NaN   
                              9            0.0  NaN  NaN             0.0  NaN   
                              10           0.0  NaN  NaN             0.0  NaN   
                              11           0.0  NaN  NaN             0.0  NaN   
                              12           0.0  NaN  NaN             0.0  NaN   
                              13           0.0  NaN  NaN             0.0  NaN   
                              14           0.0  NaN  NaN             0.0  NaN   
                              15           0.0  NaN  NaN             0.0  NaN   
                              16           0.0  NaN  NaN             0.0  NaN   
                              17           0.0  NaN  NaN             0.0  NaN   
                              18           0.0  NaN  NaN             0.0  NaN   
                              19           0.0  NaN  NaN             0.0  NaN   
                              